In [10]:
import pandas as pd
import numpy as np
import random as rd
from math import ceil,floor,factorial
from scipy.optimize import minimize

In [11]:
#Cria um dataframe com todos os times que participaram de algum campeonato mundial ou campeonatos regionais de 2018 a 2020
Times = pd.read_excel('TIMES.xlsx')
Times.set_index('Número', inplace = True)
Times.fillna('NA', inplace = True)
Times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(Times.Sigla))), index=Times.index)

#Inicializando um dicionario que ira contar as proficiencias de todos os times
proficiencia = {}

In [12]:
def liga_usada(liga,times,sigla):
    #Prepara os parametros para a computacao da verossimilhanca
    global x0,b,bnds
    x0 = list(np.random.uniform(0.5,2.5,len(times)))
    b = (0.001,5.0)
    bnds = tuple([b]*len(times))
    return (liga,times,sigla)

In [13]:
def vero(x, liga, times, sigla):
    #Computa a verossimilhanca e calcula os estimadores de maxima verossimilhanca para as proficiencias dos times
    y = 0
    proficiencia[sigla] = {}
    for time in times:
        proficiencia[sigla][time] = x[y]
        y+=1
    multiplicatorio = 1
    #Defino a constante de esquecimento k, obtida experimentalmente
    k = 0.075
    for game in range (0,liga.shape[0]):
        w = liga.iloc[game].Week
        pv = proficiencia[sigla][liga.iloc[game].Winner]
        pl = proficiencia[sigla][liga.iloc[game].Loser]
        
        #Diferenciando play-offs de fase regular
        if w>0:
            cte = 1 - (max(list(liga.Week)) - w)*k
        else:
            cte = 1+k
        multiplicatorio *= (pv/(pv+pl))**cte
    
    return -np.log(multiplicatorio)


def restricao1(x):
    return x[0]-1        

def restricao2(x):
    return sum(x)-5

def restricao3(x):
    return sum(x)-50

con1 = {'type':'eq','fun':restricao1}
con2 = {'type':'eq','fun':restricao2}
con3 = {'type':'eq','fun':restricao3}
cons = [con2]

In [14]:
def probabilidade(A,B,n,provisorio = False):
    #Calcula as probabilidades do confronto entre A e B
    A_list = []
    B_list = []
    i,k = 0,floor(n/2)
    if provisorio:
        x = salvar[A]/(salvar[A]+salvar[B])
    else:
        siglaA = list(Times[Times.Sigla==A]['Região'])[0]
        siglaB = list(Times[Times.Sigla==B]['Região'])[0]
        x = proficiencia[siglaA][A]/(proficiencia[siglaA][A]+proficiencia[siglaB][B])
    print("         {} vs {}".format(A,B))
    while (ceil(n/2)+i) <= n:
        if i == 0:
            A_list.append(x**(ceil(n/2))*(1-x)**(i))
            B_list.append(x**(i)*(1-x)**(ceil(n/2)))
        else:
            p = factorial(k)/(factorial(i)*factorial(k-i)) 
            A_list.append(p*x**(ceil(n/2))*(1-x)**(i))
            B_list.append(p*x**(i)*(1-x)**(ceil(n/2)))
        i+=1
        k+=1
    if n==1:
        print('{} tem {:.2f} % de chance de vitória'.format(A,100*sum(A_list)))
        print('Enquanto {} tem {:.2f} %.'.format(B,100*sum(B_list)))
    else:    
        print('{} tem {:.2f} % de chance de vitória.Sendo:'.format(A,100*sum(A_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*A_list[b],ceil(n/2),b))
        print('Enquanto {} tem {:.2f} %. Sendo:'.format(B,100*sum(B_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*B_list[b],ceil(n/2),b))

In [15]:
def estimar_proficiencias(regiao, championship_dic, teams_dict, initials_dict):
    #Estima as proficiencias dos times da regiao inserida
    liga,times,sigla = liga_usada(championship_dic[regiao], teams_dict[regiao], initials_dict[regiao])
    sol = minimize(vero, x0, args=(liga, times, sigla), method='SLSQP', constraints=cons , bounds=bnds)